#### Get Ram Table 

In [86]:
import os,sys # ,getpass
sys.path.insert(0,'../../libs')
from utils_pdf import get_target_pagenum,get_table_end,key_reg_match
import re
from utils import get_all_files
import PyPDF2

In [87]:
def matching_function(content):
    match_key = 'risk assessment matrix'
    if key_reg_match(content,search_text=match_key,mode='key'):
            pattern = re.compile(r'likelihood|policy response',re.I)
            # Search the text for the pattern
            res = key_reg_match(content,reg_text = pattern,mode='rgx')
            return res 
    return None

In [88]:
#PDF_folder = '/root/workspace/data/DOCs/PDF'
PDF_folder = '/root/workspace/data/DOCs/pdf_temp'
res_folder = '/root/workspace/data/ISR_Results'
all_pdfs = get_all_files(PDF_folder,end_with='.pdf')
file_names = [os.path.basename(f) for f in all_pdfs]

In [90]:
all_pdfs[:5]

['/root/workspace/data/DOCs/pdf_temp/122_2022_0.pdf',
 '/root/workspace/data/DOCs/pdf_temp/124_2022_0.pdf',
 '/root/workspace/data/DOCs/pdf_temp/132_2022_0.pdf',
 '/root/workspace/data/DOCs/pdf_temp/135_2022_0.pdf',
 '/root/workspace/data/DOCs/pdf_temp/137_2023_0.pdf']

In [91]:
results = []
for file_path in all_pdfs:
    pgn = get_target_pagenum(file_path,match_func=matching_function,return_mode='first')
    end_pgn = get_table_end(file_path,pgn,check_page_n=2)
    if not end_pgn:
        end_pgn = pgn
    results.append((file_path,pgn,end_pgn))
#print(results)


In [93]:
pd.DataFrame(results).head()

,0,1,2
0,/root/workspace/data/DOCs/pdf_temp/122_2022_0.pdf,46,47
1,/root/workspace/data/DOCs/pdf_temp/124_2022_0.pdf,35,36
2,/root/workspace/data/DOCs/pdf_temp/132_2022_0.pdf,50,50
3,/root/workspace/data/DOCs/pdf_temp/135_2022_0.pdf,34,34
4,/root/workspace/data/DOCs/pdf_temp/137_2023_0.pdf,45,45


In [94]:
pdfWriter = PyPDF2.PdfWriter()
#res_folder = '/root/workspace/data/ISR_Results'
for filename,start_pgn,end_pgn in results:
    print(filename,start_pgn,end_pgn)
    if start_pgn and end_pgn:
        with open(filename,'rb') as pdfFileObj:
            pdfReader = PyPDF2.PdfReader(pdfFileObj)
            for extract_pagenum in range(start_pgn,end_pgn+1):
                pageObj = pdfReader.pages[extract_pagenum]
                pdfWriter.add_page(pageObj)
out_name = os.path.join(res_folder,'RAM_tables_large.pdf')
pdfOutput = open(out_name, 'wb')
pdfWriter.write(pdfOutput)
pdfOutput.close()

/root/workspace/data/DOCs/pdf_temp/122_2022_0.pdf 46 47
/root/workspace/data/DOCs/pdf_temp/124_2022_0.pdf 35 36
/root/workspace/data/DOCs/pdf_temp/132_2022_0.pdf 50 50
/root/workspace/data/DOCs/pdf_temp/135_2022_0.pdf 34 34
/root/workspace/data/DOCs/pdf_temp/137_2023_0.pdf 45 45
/root/workspace/data/DOCs/pdf_temp/138_2022_0.pdf 44 45
/root/workspace/data/DOCs/pdf_temp/138_2022_1.pdf 37 38
/root/workspace/data/DOCs/pdf_temp/142_2022_0.pdf 32 33
/root/workspace/data/DOCs/pdf_temp/144_2023_0.pdf 36 36
/root/workspace/data/DOCs/pdf_temp/146_2023_0.pdf 46 46
/root/workspace/data/DOCs/pdf_temp/156_2022_0.pdf 16 16
/root/workspace/data/DOCs/pdf_temp/158_2023_0.pdf 51 51
/root/workspace/data/DOCs/pdf_temp/172_2022_0.pdf 37 38
/root/workspace/data/DOCs/pdf_temp/181_2022_0.pdf 43 43
/root/workspace/data/DOCs/pdf_temp/184_2022_0.pdf 48 49
/root/workspace/data/DOCs/pdf_temp/186_2022_0.pdf 69 69
/root/workspace/data/DOCs/pdf_temp/193_2022_0.pdf 58 58
/root/workspace/data/DOCs/pdf_temp/199_2023_0.pd

In [21]:
from llmsherpa.readers import LayoutPDFReader
from IPython.core.display import display, HTML

/tmp/ipykernel_14543/4293802776.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [22]:
llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"
pdf_url = out_name # also allowed is a file path e.g. /home/downloads/xyz.pdf
pdf_reader = LayoutPDFReader(llmsherpa_api_url)
doc = pdf_reader.read_pdf(pdf_url)

In [30]:
x  = doc.tables()[0]

In [31]:
x.block_json

{'block_class': 'cls_4',
 'block_idx': 20,
 'left': 324.6,
 'level': 5,
 'name': 'De-anchoring of inflation expectations and stagflation.',
 'page_idx': 0,
 'table_rows': [{'block_idx': 20,
   'cells': [{'cell_value': '', 'col_span': 1},
    {'cell_value': 'Near-universal automatic wage (and benefit) indexation may quickly translate de-anchored inflation expectations into wages and prices. International competitiveness would suffer, weakening the external balance and potential output. Tightening by central banks may lead to financing pressures, given elevated debt and deficit levels.',
     'col_span': 3},
    {'cell_value': 'With monetary policy levers out of the direct control of the Belgian authorities, breaking the harmful effects of a wage- price spiral would require prompt response through discretionary fiscal tightening, possibly exceptional and temporary pause to the indexation mechanism, in addition to multiannual real wage restraint, and modifications to the indexation mechan

In [32]:
import tabula

In [34]:
dfs = tabula.read_pdf(pdf_url, pages='all')

In [44]:
dfs[9]

,Source of risks,Likelihood,Expected impact if realized,Possible policy response
0,NaN,NaN,Conjunctural Risks,NaN
1,Intensification of regional,High,High,• Provide targeted fuel and
2,conflict(s). Escalation of,NaN,• Higher import bills on,food subsidies to the poor
3,Russia’s war in Ukraine or other,NaN,commodities will worsen,and vulnerable groups.
4,regional conflicts and resulting,NaN,"external position, push up",• Enhance tourism promotion
5,economic sanctions disrupt,NaN,domestic prices and,"in alternative markets,"
6,"trade (e.g., energy, food,",NaN,dampen economic activity.,including within the regional
7,"tourism, and/or critical supply",NaN,• Disrupted tourist flows and,markets.
8,"chain components),",NaN,decline in tourists from,NaN
9,"remittances, refugee flows, FDI",NaN,long-haul destinations.,NaN


In [47]:
from docx import Document
import pandas as pd

In [48]:
# Load the document
doc_path = out_name = os.path.join(res_folder,'RAM_tables.docx')
doc = Document(doc_path)

In [52]:
t = doc.tables[0]

In [66]:
t.table

_TableStyle('Table Normal') id: 139912926172304

In [59]:
table_data = []
for row in t.rows:
    row_data = [cell.text for cell in row.cells]
    table_data.append(row_data)

In [60]:
table_data

[['\nSource of Risks',
  'Relative Likelihood',
  'Time Horizon',
  '\nExpected Impact',
  '\nPolicy Response'],
 ['Global Risks',
  'Global Risks',
  'Global Risks',
  'Global Risks',
  'Global Risks'],
 ['Conjunctural Risks',
  'Conjunctural Risks',
  'Conjunctural Risks',
  'Conjunctural Risks',
  'Conjunctural Risks'],
 ['Abrupt global slowdown or recession. Global and idiosyncratic risk factors combine to cause a synchronized sharp growth slowdown, with outright recessions in some countries, spillovers through trade and financial channels, and downward pressures on some commodity prices.\nEurope: the fallout from the war in Ukraine is exacerbated by a gas shutoff by Russia, resulting in acute gas shortages and further supply disruptions, which triggers an EU recession.',
  'Medium\n\n\n\n\n\n\nHigh',
  'ST, MT',
  'High\nEnergy dependence on Russia and direct trade and financial links with Belarus, Russia, and Ukraine are relatively limited. However, indirect links and spillovers 

In [49]:
# Initialize a list to store the tables
tables_list = []

# Extracting tables from the document
for table in doc.tables:
    # Initialize a list to store each row of the table
    table_data = []
    for row in table.rows:
        row_data = [cell.text for cell in row.cells]
        table_data.append(row_data)
    # Convert each table to a DataFrame and append to the list
    df = pd.DataFrame(table_data)
    tables_list.append(df)

# Show the number of tables extracted and a preview of the first one
num_tables = len(tables_list)
first_table_preview = tables_list[0] if num_tables > 0 else None

num_tables, first_table_preview.head()

(8,
                                                    0  \
 0                                  \nSource of Risks   
 1                                       Global Risks   
 2                                 Conjunctural Risks   
 3  Abrupt global slowdown or recession. Global an...   
 4  De-anchoring of inflation expectations and sta...   
 
                           1                   2  \
 0       Relative Likelihood        Time Horizon   
 1              Global Risks        Global Risks   
 2        Conjunctural Risks  Conjunctural Risks   
 3  Medium\n\n\n\n\n\n\nHigh              ST, MT   
 4                    Medium              ST, MT   
 
                                                    3  \
 0                                  \nExpected Impact   
 1                                       Global Risks   
 2                                 Conjunctural Risks   
 3  High\nEnergy dependence on Russia and direct t...   
 4  High\nNear-universal automatic wage (and benef...

In [68]:
table_datas=[]
for table in doc.tables:
    data = []
    # Extract column headers
    headers = [cell.text.strip() for cell in table.rows[0].cells]

    # Extract row data
    for row in table.rows[1:]:
        row_data = {headers[i]: row.cells[i].text.strip() for i in range(len(row.cells))}
        data.append(row_data)
    table_datas.append(data)

In [83]:
table_datas[2][0]

{'Source of Risks': 'Abrupt global slowdown or recession 1/\n\nEMDEs: A new bout of global financial tightening, possibly combined with volatile commodity prices, leads to spiking risk premia, debt distress, widening of external imbalances, fiscal pressures, and sudden stops.\nSystemic financial instability 2/\n\n\n\nCommodity price volatility 3/\n\n\n\n\n\nDeepening geo-economic fragmentation\n4/',
 'Likelihoo': 'Medium\n\n\n\nMedium\n\n\n\nMedium\n\n\n\n\n\nHigh',
 'Expected Impact': 'High. Lower GDP growth, due to weaker investment and exports; a decline in capital inflows, leading to currency depreciation and tighter domestic credit conditions; higher poverty rate.\n\nMedium. An increase in capital outflows that lead to large exchange rate depreciation and volatility; tighter financial conditions; and upward pressures on inflation through exchange rate passthrough.\n\nMedium. The economic impacts will critically depend on whether commodity prices rise or decline, given Indonesia’s 

In [84]:
table_datas[2][1]

{'Source of Risks': 'Abrupt global slowdown or recession 1/\n\nEMDEs: A new bout of global financial tightening, possibly combined with volatile commodity prices, leads to spiking risk premia, debt distress, widening of external imbalances, fiscal pressures, and sudden stops.\nSystemic financial instability 2/\n\n\n\nCommodity price volatility 3/\n\n\n\n\n\nDeepening geo-economic fragmentation\n4/',
 'Likelihoo': 'Medium\n\n\n\nMedium\n\n\n\nMedium\n\n\n\n\n\nHigh',
 'Expected Impact': 'High. Lower GDP growth, due to weaker investment and exports; a decline in capital inflows, leading to currency depreciation and tighter domestic credit conditions; higher poverty rate.\n\nMedium. An increase in capital outflows that lead to large exchange rate depreciation and volatility; tighter financial conditions; and upward pressures on inflation through exchange rate passthrough.\n\nMedium. The economic impacts will critically depend on whether commodity prices rise or decline, given Indonesia’s 

In [85]:
table_datas[2][4]

{'Source of Risks': 'Persistently high domestic inflation, due to Medium High. Slower closing of the output gap amid global or domestic supply disruptions,\tweaker private consumption and investment; coupled with a slower-than-expected growth.\ta decline in capital inflows due to less\npositive economic prospects.',
 'Likelihoo': 'Persistently high domestic inflation, due to Medium High. Slower closing of the output gap amid global or domestic supply disruptions,\tweaker private consumption and investment; coupled with a slower-than-expected growth.\ta decline in capital inflows due to less\npositive economic prospects.',
 'Expected Impact': 'Persistently high domestic inflation, due to Medium High. Slower closing of the output gap amid global or domestic supply disruptions,\tweaker private consumption and investment; coupled with a slower-than-expected growth.\ta decline in capital inflows due to less\npositive economic prospects.',
 'Policy Recommendation': 'Use available fiscal spac